# Récupération jeu de données

L'objectif de ce notebook

Pour la documentation api Spotify



*   https://spotifypy.readthedocs.io/en/latest/api.html
*   https://github.com/spotipy-dev/spotipy/blob/master/examples/search.py
*   https://pypi.org/project/spotipy/








## Installation de l'api SPOTIFY

In [1]:
 # Installation des packages pour Spotify
#pip install spotipy 

## Import des librairies

In [2]:
from function.utils import tqdm, glob, json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Récupération des tokens

/!\ Attention /!\  
Pour pouvoir récuperer les access tokens de spotify il est impératif d'avoir un compte free ou premium.  
De plus avoir créer son APP.  
Par la suite, vous obtiendrez les client_id et client_id_secret permettant l'accès à l'API et le bon fonctionnement du code.

In [3]:
from function.function import get_token

PATH_TOKEN = "../data/token/*"
PATH_JSON = "../data/json_file/"

liste_txt_file = glob.glob(PATH_TOKEN)

GENIUS_ACCESS_TOKEN = None
SPOTIFY_client_id = None
SPOTIFY_client_secret = None

GENIUS_ACCESS_TOKEN, SPOTIFY_client_id, SPOTIFY_client_secret = get_token(liste_txt_file)


In [4]:
from function.dictionnary import get_dictionnary

dictionnaire_song = get_dictionnary(PATH_JSON, "dictionnaire_song.json")
liste_artiste = get_dictionnary(PATH_JSON, "dictionnaire_artiste.json")

In [5]:
#identifiant spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_client_id,
                                                           client_secret=SPOTIFY_client_secret))
data_artiste_titre = list()

for nom_artiste in tqdm.tqdm(liste_artiste):
    if not nom_artiste in dictionnaire_song.keys():
        results = sp.search(q=nom_artiste, limit=50) # requete de la recherche
        if 'tracks' in results and 'items' in results['tracks']:
            Info_titre = [(idx, nom_artiste, data['name']) for idx, data in enumerate(results['tracks']['items'])]
        data_artiste_titre.extend(Info_titre)

  0%|          | 0/3486 [00:00<?, ?it/s]

100%|██████████| 3486/3486 [24:13<00:00,  2.40it/s]


In [ ]:
from function.function import get_lyrics_from_genius
from function.scrapping import scrapping_find_lyrics

headers = {"User-Agent": "Mozilla/5.0"}

for idx, artist_name, song_title in tqdm.tqdm(data_artiste_titre):
    lyrics_url = get_lyrics_from_genius(song_title, artist_name, GENIUS_ACCESS_TOKEN)
    if lyrics_url:
        list_lyrics = scrapping_find_lyrics(lyrics_url, headers)
        lyrics_of_song = "\n".join(list_lyrics)
        dictionnaire_song.setdefault(artist_name, {}).setdefault(song_title, {})["lyrics"] = lyrics_of_song
    else:
        #   print(f"Lyrics de {artist_name} pour '{song_title}' not found on Genius.")
        dictionnaire_song.setdefault(artist_name, {}).setdefault(song_title, {})["lyrics"] = None

  0%|          | 0/174027 [00:00<?, ?it/s]c:\Users\Loup\anaconda3\envs\env_proj_classification_music\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.genius.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  0%|          | 0/174027 [00:02<?, ?it/s]

[Intro: Snoop Dogg]
Is this thing on? Word
[Chorus: Bruno Mars & 
Wiz Khalifa
]
So what we get drunk?
So what we smoke weed?
We're just having fun
We don’t care who sees (
Shit
)
Aye lemme get a lighter, please
So what we go out?
That's how it's supposed to be
Cause you know I’m high as fuck and I forgot one
Living young and wild and free
Keep that in there
Uh, uh-uh
[Verse 1: Wiz Khalifa]
So what I keep ‘em rolled up?
Saggin' my pants, not caring what I show
Keep it real with my niggas
Keep it player for these hoes
It look clean, don't it?
Washed it the other day, watch how you lean on it
Give me some 501 jeans on it
Roll joints bigger than King Kong's fingers
And smoke them hoes down 'til they stingers
You a class clown and if I skip for the day
I'm with your bitch smokin' grade A
Uh, hahahahahaha
[Chorus: Bruno Mars]
So what we get drunk?
So what we smoke weed?
We're just having fun
We don’t care who sees
So what we go out?
That’s how it's supposed to be
Living young and wild and fr

ZeroDivisionError: division by zero

In [ ]:
with open(PATH_JSON+"dictionnaire_song.json", "w") as json_file:
    json.dump(dictionnaire_song, json_file, indent=4)